# Task 1

Importing all the required libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, KeyedVectors
from transformers import BertTokenizer, BertModel
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading the training dataset

In [ ]:
train_data = pd.read_csv('Task-1-test-dataset3.csv')

Preprocessing for tfidf

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
processed_df = train_data.copy()

# Tfidf
def preprocess_text_tfidf(un_processed_data):
    tokens = word_tokenize(un_processed_data)
    tokenized_words = []

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    for token in tokens:
        # Checking if the word is alphabetic
        if token.isalpha() and token.lower() not in stop_words and token.lower() not in string.punctuation:
            # Converting the word to lowercase
            tokenized_words.append(lemmatizer.lemmatize(token.lower()))

    preprocessed_data = ' '.join(tokenized_words)
    return preprocessed_data




processed_df['tokenized_synopsis_tfidf'] = processed_df['plot_synopsis'].apply(preprocess_text_tfidf)

KeyError: 'plot_synopsis'

Preprocessing for word2vec

In [ ]:
# Word2Vec
def preprocess_text_word2vec(un_processed_data):
    # Tokenization
    tokens = word_tokenize(un_processed_data)

    # Removing. punctuation
    filtered_tokens_p = []
    for token in tokens:
        if token not in string.punctuation:
            filtered_tokens_p.append(token)

   # tokens = [token for token in tokens if token not in string.punctuation]

    # Lemmatization
    lemmatized_tokens = []
    for token in filtered_tokens_p:
        lemmatized_tokens.append(lemmatizer.lemmatize(token.lower()))

     # tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    # Removing stopwords
    stop_words = set(stopwords.words('english'))

    filtered_tokens_sw = []
    for token in lemmatized_tokens:
        if token.lower() not in stop_words:
            filtered_tokens_sw.append(token)

    # tokens = [token for token in tokens if token.lower() not in stop_words]

    return filtered_tokens_sw


processed_df['tokenized_synopsis_word2vec'] = processed_df['plot_synopsis'].apply(preprocess_text_word2vec)

 Sparse representation (BoW with tf*idf)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_df['tokenized_synopsis_tfidf'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

Dense static representation (Word2Vec)

In [ ]:
word2vec_model = Word2Vec(processed_df['tokenized_synopsis_word2vec'], vector_size=100, window=1, min_count=1, sg=0)

Loading the validation dataset

In [ ]:
validation_data_task_1 = pd.read_csv('./data/Task-1-validation-dataset.csv', header=None, names=['id', 'term1', 'term2', 'value'])

Calculating the cosine similarity for tfidf for validation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from google.colab import files

# Creating a vocabulary for dealing with OOV words
vocabulary = set(tfidf_df.columns)
print(len(vocabulary))
print("Index:", tfidf_df.index)


results = []

def cosine_similarity_tfidf(term1, term2, tfidf_df):
    # Checking if the terms are in the vocabulary or not
    tokens_1 = term1.split()
    tokens_2 = term2.split()

    # vectors1 = [tfidf_df[token].values if token in vocabulary else np.full(tfidf_df.shape[0], 0.01) for token in tokens1]
    vectors_1 = []
    for token in tokens_1:
        if token in vocabulary:
            vectors_1.append(tfidf_df[token].values)
        else:
            vectors_1.append(np.full(tfidf_df.shape[0], 0.01))

    # vectors2 = [tfidf_df[token].values if token in vocabulary else np.full(tfidf_df.shape[0], 0.01) for token in tokens2]

    vectors_2 = []
    for token in tokens_2:
        if token in vocabulary:
            vectors_2.append(tfidf_df[token].values)
        else:
            vectors_2.append(np.full(tfidf_df.shape[0], 0.01))

    avg_vector_1 = np.mean(vectors_1, axis=0)
    avg_vector_2 = np.mean(vectors_2, axis=0)

    avg_vector_1_2d = avg_vector_1.reshape(1, -1)
    avg_vector_2_2d = avg_vector_2.reshape(1, -1)

    return cosine_similarity(avg_vector_1_2d, avg_vector_2_2d)[0][0]

for id, (term1, term2) in zip(validation_data_task_1['id'], zip(validation_data_task_1['term1'], validation_data_task_1['term2'])):
    similarity = cosine_similarity_tfidf(term1, term2, tfidf_df)
    results.append((id, similarity))

results_df = pd.DataFrame(results, columns=['id', ''])
results_df.to_csv('10949863-Task1-method-a-validation.csv', index=False, header=False)
files.download('10949863-Task1-method-a-validation.csv')




79109
Index: RangeIndex(start=0, stop=8257, step=1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading the test dataset

In [ ]:
test_data_task_1 = pd.read_csv('./data/Task-1-test-dataset1.csv', header=None, names=['id', 'term1', 'term2', 'value'])

Calculating the cosine similarity for tfidf for test

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from google.colab import files

# Creating a vocabulary for dealing with OOV words
vocabulary = set(tfidf_df.columns)
print(len(vocabulary))

results = []

def cosine_similarity_tfidf(term1, term2, tfidf_df):
    # Checking if the terms are in the vocabulary or not
    tokens_1 = term1.split()
    tokens_2 = term2.split()

    vectors_1 = []
    for token in tokens_1:
        if token in vocabulary:
            vectors_1.append(tfidf_df[token].values)
        else:
            vectors_1.append(np.full(tfidf_df.shape[0], 0.01))


    vectors_2 = []
    for token in tokens_2:
        if token in vocabulary:
            vectors_2.append(tfidf_df[token].values)
        else:
            vectors_2.append(np.full(tfidf_df.shape[0], 0.01))

    avg_vector_1 = np.mean(vectors_1, axis=0)
    avg_vector_2 = np.mean(vectors_2, axis=0)

    avg_vector_1_2d = avg_vector_1.reshape(1, -1)
    avg_vector_2_2d = avg_vector_2.reshape(1, -1)

    return cosine_similarity(avg_vector_1_2d, avg_vector_2_2d)[0][0]

for id, (term1, term2) in zip(test_data_task_1['id'], zip(test_data_task_1['term1'], test_data_task_1['term2'])):
    similarity = cosine_similarity_tfidf(term1, term2, tfidf_df)
    results.append((id, similarity))

# Creating a DataFrame
results_df = pd.DataFrame(results, columns=['id', ''])

results_df.to_csv('10949863-Task1-method-a-test.csv', index=False, header=False)
files.download('10949863-Task1-method-a-test.csv')




79109


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading the validation dataset

In [ ]:
validation_data_task_1 = pd.read_csv('./data/Task-1-validation-dataset.csv', header=None, names=['id', 'term1', 'term2', 'value'])

Calculating the cosine similarity for word2vec for validation

In [ ]:
import pandas as pd
from google.colab import files
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity


# Creating a vocabulary for dealing with OOV words
vocabulary = set(word2vec_model.wv.key_to_index.keys())

# print(len(vocabulary))

results = []

# Extracting adjacent pairs of terms
term_pairs = [(term1, term2) for term1, term2 in zip(validation_data_task_1['term1'], validation_data_task_1['term2'])]

def cosine_similarity_word2vec(term1, term2, word2vec_model):
    # Checking if the terms are in the vocabulary or not
    tokens_1 = term1.split()
    tokens_2 = term2.split()

    vectors_1 = []
    for token in tokens_1:
        if token in vocabulary:
            vectors_1.append(word2vec_model.wv.get_vector(token))
        else:
            vectors_1.append(np.full(word2vec_model.vector_size, 0.01))

    vectors_2 = []
    for token in tokens_2:
        if token in vocabulary:
            vectors_2.append(word2vec_model.wv.get_vector(token))
        else:
            vectors_2.append(np.full(word2vec_model.vector_size, 0.01))

    avg_vector_1 = np.mean(vectors_1, axis=0)
    avg_vector_2 = np.mean(vectors_2, axis=0)

    avg_vector_1_2d = avg_vector_1.reshape(1, -1)
    avg_vector_2_2d = avg_vector_2.reshape(1, -1)

    similarity = cosine_similarity(avg_vector_1_2d, avg_vector_2_2d)[0][0]

    return similarity

# Calculate cosine similarity for each term pair
for id, (term1, term2) in zip(range(1, len(validation_data_task_1) + 1), term_pairs):
    similarity = cosine_similarity_word2vec(term1, term2, word2vec_model)
    results.append((id, similarity))

# Creating a DataFrame
results_df = pd.DataFrame(results, columns=['id', 'cosine_similarity'])

results_df.to_csv('10949863-Task1-method-b-validation.csv', index=False, header=False)
files.download('10949863-Task1-method-b-validation.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading the test dataset

In [ ]:
test_data_task_1 = pd.read_csv('./data/Task-1-test-dataset1.csv', header=None, names=['id', 'term1', 'term2', 'value'])

Calculating the cosine similarity for word2vec for test

In [ ]:
import pandas as pd
from google.colab import files
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity


# Creating a vocabulary for dealing with OOV words
vocabulary = set(word2vec_model.wv.key_to_index.keys())
print(len(vocabulary))
# to store the similarities along with the ids
results = []

# Extracting adjacent pairs of terms
term_pairs = [(term1, term2) for term1, term2 in zip(test_data_task_1['term1'], test_data_task_1['term2'])]

def cosine_similarity_word2vec(term1, term2, word2vec_model):
    # Checking if the terms are in the vocabulary or not
    tokens_1 = term1.split()
    tokens_2 = term2.split()

    # vectors1 = [word2vec_model.wv.get_vector(token) if token in vocabulary else np.full(word2vec_model.vector_size, 0.01) for token in tokens1]
    vectors_1 = []
    for token in tokens_1:
        if token in vocabulary:
            vectors_1.append(word2vec_model.wv.get_vector(token))
        else:
            vectors_1.append(np.full(word2vec_model.vector_size, 0.01))
    # vectors2 = [word2vec_model.wv.get_vector(token) if token in vocabulary else np.full(word2vec_model.vector_size, 0.01) for token in tokens2]
    vectors_2 = []
    for token in tokens_2:
        if token in vocabulary:
            vectors_2.append(word2vec_model.wv.get_vector(token))
        else:
            vectors_2.append(np.full(word2vec_model.vector_size, 0.01))
    avg_vector_1 = np.mean(vectors_1, axis=0)
    avg_vector_2 = np.mean(vectors_2, axis=0)

    avg_vector_1_2d = avg_vector_1.reshape(1, -1)
    avg_vector_2_2d = avg_vector_2.reshape(1, -1)

    similarity = cosine_similarity(avg_vector_1_2d, avg_vector_2_2d)[0][0]

    return similarity

# Calculate cosine similarity for each term pair
for id, (term1, term2) in zip(range(1, len(test_data_task_1) + 1), term_pairs):
    similarity = cosine_similarity_word2vec(term1, term2, word2vec_model)
    results.append((id, similarity))

# Create a DataFrame from the results without specifying column names
results_df = pd.DataFrame(results, columns=['id', 'cosine_similarity'])

# Save the DataFrame to a CSV file without including the index and column names
results_df.to_csv('10949863-Task1-method-b-test.csv', index=False, header=False)

# Download the file to your local machine
files.download('10949863-Task1-method-b-test.csv')



121631


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>